In [28]:
from pathlib import Path
import os
cwd = Path().resolve().parents[1]
activities_path = os.path.join(cwd , 'automagica\\activities.py')
activities_rst_path = os.path.join(cwd , 'docs\\source\\activities.rst')
github_readme_path = os.path.join(cwd , 'readme.md')

In [29]:
from pprint import pprint

with open(activities_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

categories = []

for i, line in enumerate(lines):
    if line == '"""\n': # Category
        if lines[i+1].strip(): # Not empty
            name = lines[i+1].strip()
            icon = lines[i+2].strip().replace('Icon: ','')
            if not icon:
                raise Exception
            category = {'name': name, 'icon': icon, 'activities':[]}
            categories.append(category)
            
    if line.startswith('def ') or line.startswith('    def '): # Function definition
        if lines[i-1].strip() == '@activity':
            activity = {}
            if '__init__' in line:
                if '(' in lines[i-2]:
                    parsed_class_line = lines[i-2].split('(')[0] + ':'
                else:
                    parsed_class_line = lines[i-2]
                arguments = line.split('(')[-1].replace('):','')
                func_call_raw = (parsed_class_line.replace('class ','').replace(':', '(') + arguments + ')').replace('\n','')
                activity['function_call'] = func_call_raw.replace('self, ','')

            else:
                func_call_raw = line.replace('def ','').replace(':', '').strip()
                activity['function_call'] = func_call_raw.replace('self, ','')
            activity['name'] = lines[i+1].replace('"""', '').strip()
            if not activity['name']:
                raise Exception('Activity {} should have a name'.format(activity['function_call']))

            # Description (multiline)
            activity['description'] = ''
            for k, line3 in enumerate(lines[i+3:]):
                if not line3.strip():
                    break
                activity['description'] += line3.strip()
            if not activity['description']:
                raise Exception('Activity {} should have a description'.format(activity['name']))

            # Parameters (multiline)
            activity['parameters'] = []
            activity['return'] = ''
            for l, line_param in enumerate(lines[i:]):
                if line_param.strip() == '@activity': # Till we reach next function
                    break
    
                if ':parameter' in line_param:
                    parameters = {}
                    parameters['name'] = line_param.split(':parameter')[-1].split(':')[0]
                    parameters['description'] = line_param.split(':',2)[-1]
                    
                    activity['parameters'].append(parameters)
                
                # Add return too
                if ':return:' in line_param:
                    activity['return'] = line_param.split(':',2)[-1]
            
            # Example (multiline)
            activity['example'] = ''
            for l, line4 in enumerate(lines[i:]):
                if line4.strip() == '@activity': # Till we reach next function
                    break

                if line4.strip() == ':Example:':
                    number_of_spaces = len(lines[i:][l+2]) - len(lines[i:][l+2].lstrip())
                    for m, line5 in enumerate(lines[i:][l+2:]):
                        if not line5.strip():
                            break
                        activity['example'] += line5[number_of_spaces:]
                    break
            
            # Add copy-paste example
            activity['snippet'] = ''
            for example_line in activity['example'].split('\n'):
                if example_line.strip():
                    if example_line[0] in ['>','#',' ']:
                        activity['snippet'] += example_line.replace('>>> ','') + '\n'
             
            if not activity['example']:
                raise Exception('Activity {} should have an example'.format(activity['name']))
                
            # Single lines
            for j, line2 in enumerate(lines[i:]):
                if line2.strip() == '@activity': # Till we reach next function
                    break

                if line2.strip() == 'Icon': # Icon
                    activity['icon'] = lines[i:][j+1].strip().replace('<i class="', '').replace('"></i>', '')

                if line2.strip() == 'Keywords': # Keywords
                    activity['keywords'] = lines[i:][j+1].strip().split(', ')

            category['activities'].append(activity)
            pprint(activity)
            if not activity.get('icon'):
                raise Exception('Activity {} should have an icon'.format(activity['name']))
        
import json
with open('activities.json', 'w') as f:
    json.dump(categories, f)

{'description': 'Generate random Fernet key. Fernet guarantees that a message '
                'encrypted using it cannot be manipulated or read without the '
                'key. Fernet is an implementation of symmetric (also known as '
                '“secret key”) authenticated cryptography',
 'example': '>>> # Generate a random key\n'
            '>>> generate_random_key()\n'
            "b'AYv6ZPVgnrUtHDbGZqAopRyAo9r0_UKrA2Rm3K_NjIo='\n",
 'function_call': 'generate_random_key()',
 'icon': 'las la-key',
 'keywords': ['random',
              'key',
              'fernet',
              'hash',
              'security',
              'cryptography',
              'password',
              'secure'],
 'name': 'Random key',
 'parameters': [],
 'return': ' Random key\n',
 'snippet': '# Generate a random key\ngenerate_random_key()\n'}
{'description': 'Encrypt text with (Fernet) key,',
 'example': '>>> # Generate a random key\n'
            '>>> key = generate_random_key()\n'
        

            '>>> element.click()\n',
 'function_call': 'by_class(element)',
 'icon': 'las la-times',
 'keywords': ['browser',
              'class',
              'classes',
              'element',
              'xml element by text',
              'chrome',
              'internet',
              'browsing',
              'browser',
              'surfing',
              'web',
              'webscraping',
              'www',
              'selenium',
              'crawling',
              'webtesting',
              'mozilla',
              'firefox',
              'internet explorer'],
 'name': 'Find class in browser',
 'parameters': [{'description': ' Class of element\n', 'name': ' element'}],
 'return': ' Element by class\n',
 'snippet': '# Open the browser\n'
            'browser = Chrome()\n'
            '# Go to a website\n'
            "browser.get('https://wikipedia.org')\n"
            '# Find element by class\n'
            "element = browser.by_class('search-input')\n"


 'example': '>>> # Click on a vision element, use the recorder() function to '
            'define elements\n'
            '>>> # Run the Windows calculator and try to perform the activity\n'
            ">>> run('calc.exe')\n"
            '>>> # Use the element ID found by the recorder, in this case ID '
            "'qf41'. You can also view this on automagica.id/qf41 \n"
            '>>> #  If you have a vastly different version or layout the '
            'element might not be found, use the recorder \n'
            ">>> click('qf41')\n",
 'function_call': 'click(automagica_id, delay=1)',
 'icon': 'las la-mouse-pointer',
 'keywords': ['mouse',
              'vision',
              'mouse',
              'osd',
              'overlay',
              'show',
              'display',
              'mouse automation',
              'click',
              'right click',
              'mouse button',
              'move mouse',
              'position',
              'pixel'],
 'name': '

                 'name': ' input_path'},
                {'description': ' Allow paths with an arbitrary length of 10 '
                                'characters or shorter to be emptied. Default '
                                'value is False.\n',
                 'name': ' allow_root'}],
 'return': '',
 'snippet': '# Make new folder in home directory for illustration\n'
            'testfolder = create_folder()\n'
            '# Make new text file in this folder\n'
            'text_file_location = make_text_file(output_path = testfolder)\n'
            '# Print all files in the testfolder\n'
            'get_files_in_folder(testfolder) \n'
            '# Empty the folder\n'
            'empty_folder(testfolder)\n'
            '# Check what is in the folder\n'
            'get_files_in_folder(testfolder)\n'}
{'description': 'Check whether folder exists or not, regardless if folder is '
                'empty or not.',
 'example': '>>> # Make new folder in home directory for illus

            ']\n',
 'function_call': 'get_mails(folder_name="Inbox", fields=None)',
 'icon': 'las la-mail-bulk',
 'keywords': ['outlook',
              'retrieve e-mail',
              'receive e-mails',
              'process e-mails',
              'get mails'],
 'name': 'Retrieve e-mails',
 'parameters': [{'description': ' Name of the Outlook folder, can be found '
                                'using `get_folders`.\n',
                 'name': ' folder_name'},
                {'description': ' Number of messages to retrieve\n',
                 'name': ' limit'},
                {'description': ' Fields (properties) of e-mail messages to '
                                "give, requires tupl Stadard is 'Subject', "
                                "'Body', 'SentOn' and 'SenderEmailAddress'.\n",
                 'name': ' fields'}],
 'return': ' List of dictionaries containing the e-mail messages with from, '
           'to, subject, body and html.\n',
 'snippet': 'outlook = Outloo

Exception: Activity self, folder_name="Inbox", limit=0, subject_contains="", body_contains="", sender_contains=""): should have a description

In [30]:
#%%capture cap --no-stderr

# Build activities.rst file

print('.. module:: automagica.activities')
print('')
print('Activities')
print('==========')
print('')
title = ''
for item in categories:
    print('\n')
    print(item['name'])
    underline = ''
    for char in range(0,len(item['name'])):
        underline = underline + '-'
    print(underline)
    
    for i in item['activities']:
        function = i['function_call']
        if function[0].isupper():
            class_name = i['function_call'].split('(')[0]
            class_name_rst = '.. autoclass:: ' + class_name +'\n' + '   :members:' 
            print(class_name_rst)
            break
        function_name = i['function_call'].split('(')[0]
        function_name_rst = '.. autofunction:: ' + function_name
        print(function_name_rst)

.. module:: automagica.activities

Activities



Cryptography
------------
.. autofunction:: generate_random_key
.. autofunction:: encrypt_text_with_key
.. autofunction:: decrypt_text_with_key
.. autofunction:: encrypt_file_with_key
.. autofunction:: decrypt_file_with_key
.. autofunction:: generate_key_from_password
.. autofunction:: generate_hash_from_file
.. autofunction:: generate_hash_from_text


Random
------
.. autofunction:: generate_random_number
.. autofunction:: generate_random_boolean
.. autofunction:: generate_random_name
.. autofunction:: generate_random_sentence
.. autofunction:: generate_random_address
.. autofunction:: generate_random_beep
.. autofunction:: generate_random_date
.. autofunction:: generate_date_today
.. autofunction:: generate_unique_identifier


Output
------
.. autofunction:: display_osd_message
.. autofunction:: print_console


Browser
-------
.. autoclass:: Chrome
   :members:


Credential Management
---------------------
.. autofunction:: set_credent

In [31]:
# Write to activities.rst file

with open(activities_rst_path, 'w') as f:
    f.write(cap.stdout)
    
print('Written to ' + activities_rst_path)

NameError: name 'cap' is not defined

In [32]:
#%%capture cap2 --no-stderr

# Build the readme.md

print('Process | Description')
print('------- | -----------')
title = ''
for item in categories:
    print('**' + item['name'] + '** | ‌‌ ')   
    for i in item['activities']:
        icon = i['icon']
        icon_name = str(i['icon'].split('la-')[-1])
        if icon_name not in ['html5', 'trello', 'salesforce', 'chrome', 'readme']:
            icon_name = icon_name + '-solid.svg'
        else:
            icon_name = icon_name + '.svg'
        icon_html = '<img src="https://cdn.jsdelivr.net/npm/line-awesome@1.3.0/svg/' + icon_name + '" width="20">'
        name = i['name']
        docs_link = 'https://automagica.readthedocs.io/activities.html#automagica.activities.' + i['function_call'].split('(')[0]
        linked_name = '[' + str(name) + ']' + '(' + str(docs_link) + ')'
        description = i['description']
        line = icon_html + ' ' + linked_name + ' | ' + description
        print(line)
        
print('|<img width=150/>|  ‌‌|')

Process | Description
------- | -----------
**Cryptography** | ‌‌ 
<img src="https://cdn.jsdelivr.net/npm/line-awesome@1.3.0/svg/key-solid.svg" width="20"> [Random key](https://automagica.readthedocs.io/activities.html#automagica.activities.generate_random_key) | Generate random Fernet key. Fernet guarantees that a message encrypted using it cannot be manipulated or read without the key. Fernet is an implementation of symmetric (also known as “secret key”) authenticated cryptography
<img src="https://cdn.jsdelivr.net/npm/line-awesome@1.3.0/svg/lock-solid.svg" width="20"> [Encrypt text](https://automagica.readthedocs.io/activities.html#automagica.activities.encrypt_text_with_key) | Encrypt text with (Fernet) key,
<img src="https://cdn.jsdelivr.net/npm/line-awesome@1.3.0/svg/lock-open-solid.svg" width="20"> [Decrypt text](https://automagica.readthedocs.io/activities.html#automagica.activities.decrypt_text_with_key) | Dexrypt bytes-like object to string with (Fernet) key
<img src="https:/

In [ ]:
# Replace specific part in the readme.md with the new documentation

def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

with open(github_readme_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

for line_n,line in enumerate(lines):
    if "An overview of all official Automagica activities:" in line:
        starting_index = line_n + 2
    if "## Credits" in line:
        ending_index = line_n - 1 

new_readme = [] 
for printed_line in cap2.stdout.splitlines():
    new_readme.append(printed_line + '\n')

# Replace in readme
replace_sublist(lines, lines[starting_index:ending_index], new_readme)

with open(github_readme_path, "w", encoding='utf-8') as of:
    for line in lines:
        of.write(line)

